In [18]:
#Calculating the discounted capital cost for OSeMBE
#Scenario: WP5-OPT-MIX

import pandas as pd
import numpy as np
#DiscountedCapitalInvestment
# Removing indentation SHIFT+TAB
capcost = pd.read_csv('CapitalCost_opt_mix.csv')
discount_factor = pd.read_excel('discount_rate.xlsx')
newcap = pd.read_csv('NewCapacity_opt_mix.csv')
newcap['YEAR']=newcap['YEAR'].astype(int)

# Fixed cost + TotalCapacityAnnual
# Variable Operational cost = Variable + ProductionByTechnologyAnnual
#df_merge3 = df_merge1 + df_merge2

df_merge = pd.merge(capcost, newcap, on=['REGION', 'TECHNOLOGY', 'YEAR'])
df_merge['YEAR']=df_merge['YEAR'].astype(int)
df_merge = pd.merge(df_merge, discount_factor, on=['YEAR'])
#discount_factor
df_merge

#Hash out everything: CTRL+K+C
df_merge['VALUE_r'] = df_merge['VALUE_x'] * df_merge['VALUE_y'] * df_merge['VALUE'] 
df_merge['region'] = df_merge['TECHNOLOGY'].apply(lambda x: x[:2])
countries_mod = {'AT':'Austria','BE':'Belgium','BG':'Bulgaria','CH':'Switzerland','CY':'Cyrpus','CZ':'Czech Republic','DE':'Germany','DK':'Denmark','EE':'Estonia','ES':'Spain','FI':'Finland','FR':'France','GR':'Greece','HR':'Croatia','HU':'Hungary','IE':'Ireland','IT':'Italy','LT':'Lithuania','LU':'Luxembourg','LV':'Latvia','MT':'Malta','NL':'Netherlands','NO':'Norway','PL':'Poland','PT':'Portugal','RO':'Romania','SE':'Sweden','SI':'Slovenia','SK':'Slovakia','UK':'United Kingdom','EU28':'EU28'}
countrylist = []
for i in df_merge['region']:
    countrylist.append(countries_mod[i])

df_merge['Country'] = countrylist

table = pd.pivot_table(
            df_merge,
            index=["Country"],
            columns=["YEAR"],
            values=['VALUE_r'],
            aggfunc=np.sum,
        )

table.columns = table.columns.droplevel(0)
table.index.name = None
table.to_excel('DiscountedCapitalInvestment.xlsx')

In [ ]:
#Calculating the dicounted operational cost (Fixed & Variable) for OSeMBE

import pandas as pd
import numpy as np
#DiscountedCapitalInvestment
# Removing indentation SHIFT+TAB
capcost = pd.read_csv('CapitalCost_opt_mix.csv')
discount_factor = pd.read_excel('discount_rate.xlsx')
newcap = pd.read_csv('NewCapacity_opt_mix.csv')
newcap['YEAR']=newcap['YEAR'].astype(int)

# Fixed cost + TotalCapacityAnnual
# Variable Operational cost = Variable + ProductionByTechnologyAnnual
#df_merge3 = df_merge1 + df_merge2

df_merge = pd.merge(capcost, newcap, on=['REGION', 'TECHNOLOGY', 'YEAR'])
df_merge['YEAR']=df_merge['YEAR'].astype(int)
df_merge = pd.merge(df_merge, discount_factor, on=['YEAR'])
#discount_factor
df_merge

#Hash out everything: CTRL+K+C
df_merge['VALUE_r'] = df_merge['VALUE_x'] * df_merge['VALUE_y'] * df_merge['VALUE'] 
df_merge['region'] = df_merge['TECHNOLOGY'].apply(lambda x: x[:2])
countries_mod = {'AT':'Austria','BE':'Belgium','BG':'Bulgaria','CH':'Switzerland','CY':'Cyrpus','CZ':'Czech Republic','DE':'Germany','DK':'Denmark','EE':'Estonia','ES':'Spain','FI':'Finland','FR':'France','GR':'Greece','HR':'Croatia','HU':'Hungary','IE':'Ireland','IT':'Italy','LT':'Lithuania','LU':'Luxembourg','LV':'Latvia','MT':'Malta','NL':'Netherlands','NO':'Norway','PL':'Poland','PT':'Portugal','RO':'Romania','SE':'Sweden','SI':'Slovenia','SK':'Slovakia','UK':'United Kingdom','EU28':'EU28'}
countrylist = []
for i in df_merge['region']:
    countrylist.append(countries_mod[i])

df_merge['Country'] = countrylist

table = pd.pivot_table(
            df_merge,
            index=["Country"],
            columns=["YEAR"],
            values=['VALUE_r'],
            aggfunc=np.sum,
        )

table.columns = table.columns.droplevel(0)
table.index.name = None
table.to_excel('DiscountedCapitalInvestment.xlsx')

In [ ]:
#AnnualFixedOperatingCost
fixcost = df[df['PARAM']=='FixedCost']
fixcost = fixcost[['TECHNOLOGY', 'VALUE']].copy()
newcap = res_df[res_df['NAME'] == 'NewCapacity']
df_merge = pd.merge(fixcost, newcap, on=['TECHNOLOGY'])
df_merge['YEAR'] = df_merge['YEAR'].astype(int)
df_merge['VALUE_r'] = df_merge['VALUE_x'] * df_merge['VALUE_y']
df_merge = df_merge.drop(['VALUE_x', 'VALUE_y'], axis=1)
df_merge.rename(columns = {'VALUE_r':'VALUE' }, inplace = True)
namelist = []
for i in range(0, len(df_merge)):
    namelist.append('AnnualFixedOperatingCost')
df_merge['NAME'] = namelist
res_df = pd.concat([res_df, df_merge], ignore_index=True, sort=False)


#AnnualVariableOperatingCost
varcost = df[df['PARAM']=='VariableCost']
varcost = varcost[['MODE_OF_OPERATION', 'TECHNOLOGY', 'YEAR', 'VALUE']].copy()
varcost['YEAR'] = varcost['YEAR'].astype(int)
varcost['MODE_OF_OPERATION'] = varcost['MODE_OF_OPERATION'].astype(int)
df_merge = pd.merge(roa, varcost, on=['MODE_OF_OPERATION', 'TECHNOLOGY', 'YEAR'])
df_merge = pd.merge(df_merge, ys_f, on=['TIMESLICE', 'YEAR'])
df_merge['VALUE_r'] = df_merge['VALUE'] * df_merge['VALUE_x'] * df_merge['VALUE_y']
df_merge['VALUE_r'] = df_merge['VALUE_r'].astype(float)
df_merge = df_merge.drop(['VALUE_x', 'VALUE', 'VALUE_y'], axis=1)
df_merge.rename(columns = {'VALUE_r':'VALUE'}, inplace = True)
df_merge = df_merge.groupby(['YEAR', 'TECHNOLOGY', 'NAME']).sum()
df_merge=df_merge.reset_index()
namelist = []
for i in range(0, len(df_merge)):
    namelist.append('AnnualVariableOperatingCost')
df_merge['NAME'] = namelist
res_df = pd.concat([res_df, df_merge], ignore_index=True, sort=False)


#DiscountedOperatingCost
avc = res_df[res_df['NAME']=='AnnualVariableOperatingCost'].copy()
afc = res_df[res_df['NAME']=='AnnualFixedOperatingCost'].copy()
afc = afc[['TECHNOLOGY', 'YEAR', 'VALUE']].copy()
df_merge = pd.merge(afc, avc, on=['TECHNOLOGY', 'YEAR'])
df_merge = pd.merge(df_merge, dr_f, on=['TECHNOLOGY', 'YEAR'])
df_merge['VALUE_r'] = (df_merge['VALUE_x'] + df_merge['VALUE_y']) * ( 1 / (1 + df_merge['VALUE'])**(df_merge['YEAR'] - min(df_merge['YEAR'])))
df_merge = df_merge.drop(['VALUE_x', 'VALUE', 'VALUE_y'], axis=1)
df_merge.rename(columns = {'VALUE_r':'VALUE' }, inplace = True)
namelist = []
for i in range(0, len(df_merge)):
    namelist.append('DiscountedOperatingCost')
df_merge['NAME'] = namelist
res_df = pd.concat([res_df, df_merge], ignore_index=True, sort=False)